In [1]:
import pandas as pd
import os
import numpy as np
import json

### Preparando tabelas oriundas da ANP

In [2]:
df=pd.read_excel('tabela-de-pocos.xlsx')

In [3]:
len(df)

30465

In [4]:
df.columns

Index(['POCO', 'CADASTRO', 'OPERADOR', 'POCO_OPERADOR', 'ESTADO', 'BACIA',
       'BLOCO', 'SIG_CAMPO', 'CAMPO', 'TERRA_MAR', 'POCO_POS_ANP', 'TIPO',
       'CATEGORIA', 'RECLASSIFICACAO', 'SITUACAO', 'INICIO', 'TERMINO',
       'CONCLUSAO', 'TITULARIDADE', 'LATITUDE_BASE_4C', 'LONGITUDE_BASE_4C',
       'LATITUDE_BASE_DD', 'LONGITUDE_BASE_DD', 'DATUM_HORIZONTAL',
       'TIPO_DE_COORDENADA_DE_BASE', 'DIRECAO', 'PROFUNDIDADE_VERTICAL_M',
       'PROFUNDIDADE_SONDADOR_M', 'PROFUNDIDADE_MEDIDA_M',
       'REFERENCIA_DE_PROFUNDIDADE', 'MESA_ROTATIVA', 'COTA_ALTIMETRICA_M',
       'LAMINA_D_AGUA_M', 'DATUM_VERTICAL', 'UNIDADE_ESTRATIGRAFICA',
       'GEOLOGIA_GRUPO_FINAL', 'GEOLOGIA_FORMACAO_FINAL',
       'GEOLOGIA_MEMBRO_FINAL', 'CDPE', 'AGP', 'PC', 'PAG',
       'PERFIS_CONVENCIONAIS', 'DURANTE_PERFURACAO', 'PERFIS_DIGITAIS',
       'PERFIS_PROCESSADOS', 'PERFIS_ESPECIAIS', 'AMOSTRA_LATERAL', 'SISMICA',
       'TABELA_TEMPO_PROFUNDIDADE', 'DADOS_DIRECIONAIS', 'TESTE_A_CABO',
       'TES

In [5]:
df.drop(columns = ['POCO_POS_ANP', 'TIPO', 'INICIO', 'TERMINO', 'CONCLUSAO', 'TITULARIDADE', 
                   'LATITUDE_BASE_4C', 'LONGITUDE_BASE_4C', 'LATITUDE_BASE_DD', 'LONGITUDE_BASE_DD', 
                   'DATUM_HORIZONTAL', 'TIPO_DE_COORDENADA_DE_BASE', 'DIRECAO', 
                   'PROFUNDIDADE_VERTICAL_M', 'PROFUNDIDADE_SONDADOR_M', 'PROFUNDIDADE_MEDIDA_M',
                   'REFERENCIA_DE_PROFUNDIDADE', 'MESA_ROTATIVA', 'COTA_ALTIMETRICA_M',
                   'DATUM_VERTICAL', 'UNIDADE_ESTRATIGRAFICA', 'CDPE', 'AGP', 'PC', 'PAG',
                   'PERFIS_CONVENCIONAIS', 'DURANTE_PERFURACAO', 'PERFIS_DIGITAIS', 'PERFIS_PROCESSADOS', 
                   'PERFIS_ESPECIAIS', 'AMOSTRA_LATERAL', 'SISMICA', 'TABELA_TEMPO_PROFUNDIDADE', 
                   'DADOS_DIRECIONAIS', 'TESTE_A_CABO', 'TESTE_DE_FORMACAO', 'CANHONEIO', 'TESTEMUNHO', 
                   'GEOQUIMICA', 'SIG_SONDA', 'NOM_SONDA', 'ATINGIU_PRESAL', 'DHA_ATUALIZACAO'
                  ], inplace=True)

In [6]:
df['CADASTRO'].nunique()

30465

#### Avaliando instâncias de Poço

In [7]:
df['CADASTRO'].max()

901210398600

In [8]:
df['CADASTRO'].min()

8115018892

In [9]:
def _generate_well_codes(x):
    return 'POCO_'+str(x).zfill(13)

df['COD_POCO']=df['CADASTRO'].apply(lambda x: _generate_well_codes(x))

#### Avaliando instâncias de Bacia

In [11]:
df['BACIA'].nunique()

37

In [12]:
codigosBacias = {}
i=1
for bacia in sorted(df['BACIA'].unique()):
    codigoBacia='BACIA_'+str(i).zfill(2)
    codigosBacias[codigoBacia]=bacia
    i+=1    

In [13]:
df[df['BACIA'].isna()]

,POCO,CADASTRO,OPERADOR,POCO_OPERADOR,ESTADO,BACIA,BLOCO,SIG_CAMPO,CAMPO,TERRA_MAR,CATEGORIA,RECLASSIFICACAO,SITUACAO,LAMINA_D_AGUA_M,GEOLOGIA_GRUPO_FINAL,GEOLOGIA_FORMACAO_FINAL,GEOLOGIA_MEMBRO_FINAL,COD_POCO


In [14]:
with open('codigos_bacias.json', 'w') as f:
    json.dump(codigosBacias, f)

In [15]:
def _codigo_bacia(x, codigosBacias):
    for key, value in codigosBacias.items():
        if x==value:
            return(key)

In [16]:
df['COD_BACIA']=df['BACIA'].apply(lambda x: _codigo_bacia(x, codigosBacias))

#### Avaliando instâncias de Campos

In [18]:
df['CAMPO'].nunique()

535

In [19]:
df['SIG_CAMPO'].nunique()

497

In [20]:
def _uniform_field_names(x):
    if isinstance(x, str):
        x=x.replace('VAZA BARRIS','VAZA-BARRIS')
        x=x.replace('QUIRICO','QUIRICÓ')
        x=x.replace(' (Licitado)','')
        x=x.upper()
    return x

In [21]:
df['CAMPO_CORRIGIDO']=df['CAMPO'].apply(lambda x: _uniform_field_names(x))

In [22]:
df_campos = df[['SIG_CAMPO', 'CAMPO_CORRIGIDO', 'COD_BACIA']]
df_campos=df_campos.drop_duplicates().dropna()
df_campos.reset_index(drop=True, inplace=True)

In [23]:
codigosCampos = {}
for bacia in sorted(df_campos['COD_BACIA'].unique()):
    c=1
    for campo in sorted(df_campos[df_campos['COD_BACIA']==bacia]['CAMPO_CORRIGIDO'].unique()):
        codigoCampo=bacia.replace('BACIA','CAMPO')+str(c).zfill(4)
        codigosCampos[codigoCampo]=campo
        c+=1 

In [24]:
with open('codigos_campos.json', 'w') as f:
    json.dump(codigosCampos, f)

In [25]:
def _codigo_campo(campo, bacia, codigosCampos):
    for key, value in codigosCampos.items():
        if bacia=='BACIA_'+key[6:8] and campo==value:
            return(key)

In [26]:
df['COD_CAMPO']=df.apply(lambda x: _codigo_campo(x['CAMPO_CORRIGIDO'], x['COD_BACIA'], codigosCampos), axis=1)

#### Avaliando instâncias de unidades litoestratigráficas

In [27]:
df['GEOLOGIA_MEMBRO_FINAL'].unique()

array([nan, 'Ilha Bela', 'Tamoios', 'Gomo', 'Mucuri', 'Santiago',
       'Capianga', 'Caruaçu', 'Catu', 'Maresias', 'Geribá', 'Upanema',
       'Pitanga', 'Boipeba', 'Sapucari', 'Búzios', 'Quissamã',
       'Carmópolis', 'Tauá', 'Grussaí', 'Sequência 3', 'Siri', 'Galinhos',
       'São Tomé', 'Cazumba'], dtype=object)

In [28]:
df['GEOLOGIA_FORMACAO_FINAL'].unique()

array([nan, 'Barra Velha', 'Serraria', 'Itaparica', 'São Mateus', 'Ariri',
       'Piçarras', 'Sergi', 'Marfim', 'Juruá', 'Itapema', 'Pendência',
       'Embasamento', 'Quissamã', 'Carapebus', 'Itajaí-Açu', 'Ererê',
       'Ubatuba', 'Urucutuca', 'Candeias', 'Mariricu', 'Longá', 'Pojuca',
       'Açu', 'Poti', 'Aliança', 'Calumbi', 'Barra de Itiuba',
       'Maracangalha', 'Oriximiná', 'Macabu', 'Água Grande', 'Cabiúnas',
       'Coqueiro Seco', 'Marambaia', 'Salvador', 'Cabeças', 'Pimenteiras',
       'Curiri', 'Alagamar', 'Coqueiros', 'Itaim', 'Cotinguiba',
       'Muribeca', 'Bananeiras', 'Cricaré', 'Penedo', 'Atafona',
       'Pescada', 'Algodões', 'Retiro', 'Guarujá', 'Taquipe', 'Macaé',
       'Bauxi', 'Camboriú', 'Motuca', 'Namorado', 'Paracuru',
       'Rio Pitanga', 'Lagoa Feia', 'Ipu', 'Emboré', 'Nova Olinda',
       'Regência', 'Batinga', 'Maceió', 'Riachão', 'Jutaí', 'Itaituba',
       'Afligidos'], dtype=object)

In [29]:
df['GEOLOGIA_GRUPO_FINAL'].unique()

array([nan, 'Guaratiba', 'Perucaba', 'Santo Amaro', 'Barra Nova',
       'Brotas', 'Ilhas', 'Tefé', 'Areia Branca', 'Macaé', 'Campos',
       'Frade', 'Urupadi', 'Espírito Santo', 'Nativo', 'Canindé', 'Apodi',
       'Piaçabuçu', 'Coruripe', 'Curuá', 'Lagoa Feia', 'Itamambuca',
       'Cuiabá', 'Sergipe', 'Paranoá', 'Camamu', 'Camburi', 'Espinhaço',
       'Purus', 'Jangada', 'Balsas', 'Macaúbas', 'Serra Grande',
       'Tapajós', 'Igreja Nova', 'Afligidos'], dtype=object)

In [30]:
def _codigo_litoestratigrafia(x, codigoMapper):
    for key, value in codigoMapper.items():
        if x==value:
            return(key)

In [31]:
codigosGrupos = {}
g=1
for grupo in df['GEOLOGIA_GRUPO_FINAL'].unique():
    if isinstance(grupo, str):
        codigoGrupo='GRUPO_'+str(g).zfill(2)
        codigosGrupos[codigoGrupo]=grupo
        g+=1

In [32]:
df['COD_GRUPO']=df['GEOLOGIA_GRUPO_FINAL'].apply(lambda x: _codigo_litoestratigrafia(x, codigosGrupos))

In [33]:
codigosFormacoes = {}
codigosFormacaoGrupo = {}
f=1
for formacao in df['GEOLOGIA_FORMACAO_FINAL'].unique():
    if isinstance(formacao, str):
        codigoFormacao='FORMACAO_'+str(f).zfill(3)
        codigosFormacoes[codigoFormacao]=formacao
        f+=1
        for codigoGrupo in df[df['GEOLOGIA_FORMACAO_FINAL']==formacao]['COD_GRUPO'].unique():
            if isinstance(codigoGrupo, str):
                codigosFormacaoGrupo[codigoFormacao]=codigoGrupo

In [34]:
df['COD_FORMACAO']=df['GEOLOGIA_FORMACAO_FINAL'].apply(lambda x: _codigo_litoestratigrafia(x, codigosFormacoes))

In [35]:
codigosMembros = {}
codigosMembroFormacao = {}
m=1
for membro in df['GEOLOGIA_MEMBRO_FINAL'].unique():
    if isinstance(membro, str):
        codigoMembro='MEMBRO_'+str(m).zfill(2)
        codigosMembros[codigoMembro]=membro
        m+=1
        for codigoFormacao in df[df['GEOLOGIA_MEMBRO_FINAL']==membro]['COD_FORMACAO'].unique():
            if isinstance(codigoFormacao, str):
                codigosMembroFormacao[codigoMembro]=codigoFormacao

In [36]:
df['COD_MEMBRO']=df['GEOLOGIA_MEMBRO_FINAL'].apply(lambda x: _codigo_litoestratigrafia(x, codigosMembros))

In [37]:
with open('formacao_grupo.json', 'w') as f:
    json.dump(codigosFormacaoGrupo, f)
with open('membro_formacao.json', 'w') as f:
    json.dump(codigosMembroFormacao, f)
with open('codigos_membro.json', 'w') as f:
    json.dump(codigosMembros, f)
with open('codigos_formacao.json', 'w') as f:
    json.dump(codigosFormacoes, f)
with open('codigos_grupo.json', 'w') as f:
    json.dump(codigosGrupos, f)

In [38]:
df.drop(columns=['CADASTRO', 'OPERADOR', 'BLOCO', 'TERRA_MAR', 'CATEGORIA', 'RECLASSIFICACAO', 'SITUACAO', 'LAMINA_D_AGUA_M'], inplace=True)

In [39]:
df[295:305]

,POCO,POCO_OPERADOR,ESTADO,BACIA,SIG_CAMPO,CAMPO,GEOLOGIA_GRUPO_FINAL,GEOLOGIA_FORMACAO_FINAL,GEOLOGIA_MEMBRO_FINAL,COD_POCO,COD_BACIA,CAMPO_CORRIGIDO,COD_CAMPO,COD_GRUPO,COD_FORMACAO,COD_MEMBRO
295,8-ABL-82H-RJS,8ABL82HRJS,RJ,Campos,ABL,ALBACORA LESTE,Campos,Carapebus,NaN,POCO_0074281027798,BACIA_09,ALBACORA LESTE,CAMPO_090003,GRUPO_10,FORMACAO_014,None
296,4-BRSA-1281-RN,4SVM10RN,RN,Potiguar,SVM,SERRA VERMELHA,NaN,Embasamento,NaN,POCO_0072100027870,BACIA_26,SERRA VERMELHA,CAMPO_260100,None,FORMACAO_012,None
297,7-MXL-18H-SPS,7MXL18HSPS,SP,Santos,MXL,MEXILHÃO,Frade,Itajaí-Açu,Ilha Bela,POCO_0086316027843,BACIA_29,MEXILHÃO,CAMPO_290023,GRUPO_11,FORMACAO_015,MEMBRO_01
298,1-BRSA-1269-AM,1RJP1AM,AM,Amazonas,NaN,NaN,Urupadi,Ererê,NaN,POCO_0014030027801,BACIA_04,NaN,None,GRUPO_12,FORMACAO_016,None
299,9-BRSA-1284-RJS,9RJS730,RJ,Santos,ATP,ATAPU,Guaratiba,Itapema,NaN,POCO_0074316027881,BACIA_29,ATAPU,CAMPO_290002,GRUPO_01,FORMACAO_010,None
300,3-KPGL-4-SPS,3KPLG4SPS,SP,Santos,GOI,GOIÁ,Frade,Itajaí-Açu,NaN,POCO_0086316027867,BACIA_29,GOIÁ,CAMPO_290016,GRUPO_11,FORMACAO_015,None
301,3-BRSA-1216DA-SPS,3SPS104DA,SP,Santos,BAC,BACALHAU,Guaratiba,Barra Velha,NaN,POCO_0086316027781,BACIA_29,BACALHAU,CAMPO_290004,GRUPO_01,FORMACAO_001,None
302,3-BRSA-1279-SE,3CP1926SE,SE,Sergipe,CP,CARMÓPOLIS,NaN,Embasamento,NaN,POCO_0090120027849,BACIA_30,CARMÓPOLIS,CAMPO_300018,None,FORMACAO_012,None
303,7-DJ-886D-BA,7DJ886DBA,BA,Recôncavo,DJ,DOM JOÃO,Brotas,Sergi,NaN,POCO_0020240027908,BACIA_27,DOM JOÃO,CAMPO_270028,GRUPO_05,FORMACAO_007,None
304,7-FSR-61-ES,7FSR61ES,ES,Espírito Santo,FSRL,FAZENDA SÃO RAFAEL,Barra Nova,São Mateus,NaN,POCO_0034270027886,BACIA_12,FAZENDA SÃO RAFAEL,CAMPO_120025,GRUPO_04,FORMACAO_004,None


In [40]:
df.to_excel('dados_anp.xlsx')